In [1]:
import os
import cv2
from tqdm import tqdm
import random
import numpy as np
import json
import matplotlib.pyplot as plt

In [2]:
os.chdir('/home/gptrapletti/ds/satellite-multiclass-segm')

In [3]:
image_filepaths = sorted([os.path.join('data/images', filename) for filename in os.listdir(os.path.join('data/images'))])
mask_filepaths = sorted([os.path.join('data/masks', filename) for filename in os.listdir(os.path.join('data/masks'))])

len(image_filepaths), len(mask_filepaths)

(400, 400)

In [17]:
# Check image sizes
sizes = []
for i in tqdm(range(len(image_filepaths))):
    image = cv2.imread(image_filepaths[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mask = cv2.imread(mask_filepaths[i])
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
    size = (image.shape, mask.shape)
    sizes.append(size)
    
set(sizes)

100%|██████████| 400/400 [04:13<00:00,  1.58it/s]


{((4000, 6000, 3), (4000, 6000, 3))}

All the images have the same size.

In [4]:
category_colors = {
    "unlabeled": (0, 0, 0),
    "paved-area": (128, 64, 128),
    "dirt": (130, 76, 0),
    "grass": (0, 102, 0),
    "gravel": (112, 103, 87),
    "water": (28, 42, 168),
    "rocks": (48, 41, 30),
    "pool": (0, 50, 89),
    "vegetation": (107, 142, 35),
    "roof": (70, 70, 70),
    "wall": (102, 102, 156),
    "window": (254, 228, 12),
    "door": (254, 148, 12),
    "fence": (190, 153, 153),
    "fence-pole": (153, 153, 153),
    "person": (255, 22, 96),
    "dog": (102, 51, 0),
    "car": (9, 143, 150),
    "bicycle": (119, 11, 32),
    "tree": (51, 51, 0),
    "bald-tree": (190, 250, 190),
    "ar-marker": (112, 150, 146),
    "obstacle": (2, 135, 115),
    "conflicting": (255, 0, 0)
}

For each class, look for 3 random images showing that class, turn everything to black and then plot.

In [ ]:
random_idxs = list(range(len(mask_filepaths)))

category_examples = {}
for category in tqdm(category_colors):
    random.shuffle(random_idxs)
    example_idxs = []
    n = 0
    for i in random_idxs:
        n += 1
        mask = cv2.imread(mask_filepaths[i])
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
        has_color = np.any(np.all(mask == category_colors[category], axis=2)) # if mask == [128, 64, 128] across color channels
        # If it has at least a pixel with the object class color
        if has_color:
            example_idxs.append(i)
        if len(example_idxs) == 3: # examples found
            category_examples[category] = example_idxs           
            break
        elif i == random_idxs[-1]: # no examples found
            break

In [5]:
# # Saver dict
# with open('data/category_examples.json', 'w') as file:
#     json.dump(category_examples, file)

# Load dict
with open('data/category_examples.json', 'r') as file:
    category_example_idxs = json.load(file)

In [6]:
absent_categories = set(category_colors) - set(category_example_idxs)
print(f'Categories absent in the images are: {list(absent_categories)}')

Categories absent in the images are: ['conflicting']


In [83]:
# For each category, find 3 images with objects of that category, turn everything to background and save plots.
categories = sorted(list(category_example_idxs))

for category in tqdm(categories):
    fig, axs = plt.subplots(1, 3, figsize=(20, 6))
    for i, idx in enumerate(category_example_idxs[category]):
        # Load image
        image = cv2.imread(image_filepaths[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (400, 300))
        # Load mask
        mask = cv2.imread(mask_filepaths[idx])
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
        mask = cv2.resize(mask, (400, 300), interpolation=cv2.INTER_NEAREST)
        # Keep only category of interest in the mask
        filter = np.all(mask == category_colors[category], axis=2)
        filter = np.stack([filter]*3, axis=-1)
        # Filter image
        image_filtered = image * filter
        # Plot        
        axs[i].imshow(image_filtered)
        axs[i].axis('off')
        axs[i].set_title(category)
    
    fig.tight_layout()
    fig.savefig(f'plots/{category}.png')
    plt.close(fig)   

100%|██████████| 23/23 [00:48<00:00,  2.13s/it]


In [ ]:
# Count total number of pixels for each category
# (Actually it's not the total numer of pixels because the
# images get resized and background is not counted).
pixel_counts = {key: 0 for key in category_colors}

for i in tqdm(range(len(mask_filepaths))):
    # Load mask
    mask = cv2.imread(mask_filepaths[i])
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
    mask = cv2.resize(mask, (400, 300), interpolation=cv2.INTER_NEAREST)
    # Init empty count dict for the mask
    mask_pixel_counts = {key: 0 for key in category_colors}
    # For each category count n pixels and add to mask dict
    for category in mask_pixel_counts:
        count = np.sum(np.all(mask == category_colors[category], axis=2))
        mask_pixel_counts[category] = count.item() # as int, not np.array
    # Add mask dict values to main dict
    for key in mask_pixel_counts:
        pixel_counts[key] += mask_pixel_counts[key]

100%|██████████| 400/400 [02:44<00:00,  2.43it/s]


In [7]:
# Save
# with open('data/category_pixel_counts.json', 'w') as file:
#     json.dump(pixel_counts, file)

# Load
with open('data/category_pixel_counts.json', 'r') as file:
    pixel_counts = json.load(file)

In [8]:
# Get percentage of pixels for each category on the total number of pixels
# belonging to some category.
n_tot_pixels = np.sum([pixel_counts[category] for category in pixel_counts])
pixel_percs = {category: np.round(pixel_counts[category]/n_tot_pixels, 4) for category in pixel_counts}

sorted(list(pixel_percs.items()), key=lambda x: -x[1])

[('paved-area', 0.3766),
 ('grass', 0.1994),
 ('roof', 0.0735),
 ('gravel', 0.0729),
 ('vegetation', 0.0709),
 ('obstacle', 0.0354),
 ('dirt', 0.0319),
 ('wall', 0.0268),
 ('water', 0.0221),
 ('tree', 0.0205),
 ('bald-tree', 0.0133),
 ('person', 0.0105),
 ('fence', 0.0096),
 ('car', 0.0078),
 ('rocks', 0.0072),
 ('pool', 0.0064),
 ('window', 0.0056),
 ('unlabeled', 0.0043),
 ('ar-marker', 0.0023),
 ('bicycle', 0.0022),
 ('fence-pole', 0.0005),
 ('door', 0.0003),
 ('dog', 0.0001),
 ('conflicting', 0.0)]

In [14]:
# Relabeling
regrouping = {
    'background': ['ar-marker', 'obstacle', 'car', 'unlabeled', 'bicycle', 'dog', 'conflicting', 'fence', 'pool', 'fence-pole'],
    'ground': ['paved-area', 'gravel', 'dirt', 'rocks'],
    'vegetation': ['grass', 'vegetation', 'tree', 'bald-tree'],
    'buildings': ['roof', 'wall', 'window', 'door'],
    'water': ['water'],
    'person' : ['person']
}

In [15]:
# New category percentages
new_pixel_percs = {}
for new_category in regrouping:
    new_category_pixel_count = []
    for category in regrouping[new_category]:
        new_category_pixel_count.append(pixel_counts[category])
    new_pixel_percs[new_category] = np.round(np.sum(new_category_pixel_count) / n_tot_pixels, 4)
    
sorted(list(new_pixel_percs.items()), key=lambda x:-x[1])       

[('ground', 0.4886),
 ('vegetation', 0.3041),
 ('buildings', 0.1062),
 ('background', 0.0686),
 ('water', 0.0221),
 ('person', 0.0105)]

In [16]:
# Original colors to map to the new categories
new_category_colors = {}

for new_category in regrouping:
    new_category_color_ls = []
    for category in regrouping[new_category]:
        new_category_color_ls.append(category_colors[category])
    new_category_colors[new_category] = new_category_color_ls
    
for item in new_category_colors.items():
    print(item)

('background', [(112, 150, 146), (2, 135, 115), (9, 143, 150), (0, 0, 0), (119, 11, 32), (102, 51, 0), (255, 0, 0), (190, 153, 153), (0, 50, 89), (153, 153, 153)])
('ground', [(128, 64, 128), (112, 103, 87), (130, 76, 0), (48, 41, 30)])
('vegetation', [(0, 102, 0), (107, 142, 35), (51, 51, 0), (190, 250, 190)])
('buildings', [(70, 70, 70), (102, 102, 156), (254, 228, 12), (254, 148, 12)])
('water', [(28, 42, 168)])
('person', [(255, 22, 96)])


The "background" class should be set to 0, to really become the background.